لتشغيل دفاتر الملاحظات التالية، إذا لم تقم بذلك بعد، تحتاج إلى تعيين مفتاح openai داخل ملف .env باسم `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

بعد ذلك، سنقوم بتحميل فهرس التضمين في إطار بيانات Pandas. يتم تخزين فهرس التضمين في ملف JSON يسمى `embedding_index_3m.json`. يحتوي فهرس التضمين على التضمينات لكل من نصوص يوتيوب حتى أواخر أكتوبر 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

بعد ذلك، سنقوم بإنشاء دالة تسمى `get_videos` التي ستبحث في فهرس التضمين عن الاستعلام. ستُرجع الدالة أفضل 5 مقاطع فيديو هي الأكثر تشابهًا مع الاستعلام. تعمل الدالة كما يلي:

1. أولاً، يتم إنشاء نسخة من فهرس التضمين.
2. بعد ذلك، يتم حساب التضمين للاستعلام باستخدام واجهة برمجة تطبيقات التضمين من OpenAI.
3. ثم يتم إنشاء عمود جديد في فهرس التضمين يسمى `similarity`. يحتوي عمود `similarity` على التشابه الكوني بين تضمين الاستعلام وتضمين كل مقطع فيديو.
4. بعد ذلك، يتم تصفية فهرس التضمين بواسطة عمود `similarity`. يتم تصفية فهرس التضمين ليشمل فقط مقاطع الفيديو التي لها تشابه كوني أكبر من أو يساوي 0.75.
5. أخيرًا، يتم فرز فهرس التضمين حسب عمود `similarity` ويتم إرجاع أفضل 5 مقاطع فيديو.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

هذه الدالة بسيطة جدًا، فهي فقط تطبع نتائج استعلام البحث.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. أولاً، يتم تحميل فهرس التضمين في إطار بيانات Pandas.
2. بعد ذلك، يُطلب من المستخدم إدخال استعلام.
3. ثم يتم استدعاء دالة `get_videos` للبحث في فهرس التضمين عن الاستعلام.
4. أخيرًا، يتم استدعاء دالة `display_results` لعرض النتائج على المستخدم.
5. ثم يُطلب من المستخدم إدخال استعلام آخر. تستمر هذه العملية حتى يدخل المستخدم `exit`.

![](../../../../translated_images/ar/notebook-search.1e320b9c7fcbb0bc.png)

سيُطلب منك إدخال استعلام. أدخل استعلامًا واضغط إدخال. ستُرجع التطبيق قائمة بالفيديوهات ذات الصلة بالاستعلام. كما ستُرجع التطبيق رابطًا إلى المكان في الفيديو حيث يقع جواب السؤال.

إليك بعض الاستعلامات لتجربتها:

- ما هو Azure Machine Learning؟
- كيف تعمل الشبكات العصبية الالتفافية؟
- ما هي الشبكة العصبية؟
- هل يمكنني استخدام دفاتر Jupyter مع Azure Machine Learning؟
- ما هو ONNX؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الهامة، يُنصح بالاعتماد على الترجمة البشرية المهنية. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
